In [1]:
#dependencies

#Import linear algebra and data manipulation
import numpy as np
import pandas as pd


#Import machine learning
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import math

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#read the dataset
listings_df = pd.read_csv('./listings.csv')

#find number of listings and number of hosts
listings_number = listings_df['id'].count()
hosts_number = len(listings_df['host_id'].unique())

print('listings.csv dataset contains information on %d listings provided by %d hosts.' % (listings_number, 
                                                                                          hosts_number))

listings.csv dataset contains information on 3723 listings provided by 2732 hosts.


In [3]:
#Helper functions for dataset cleaning
def get_host_since_year(row):
    ''' Get year from a date represented as a string '''
    try:
        host_since = int(row['host_since'].split('-')[0])
    except:
        host_since = np.nan
    return host_since

def get_val_from_list(row, column_name, value):
    ''' Fill in dummy column for values '''
    val = 0.0
    try:
        vals = row[column_name].replace('[', '').replace("'", '').replace("]", '').replace('"', '').replace('{', '').replace('}', '').split(',')
        if value in vals:
            val = 1.0
    except:
        val = 0.0
    return val

def split_list_into_columns(df, column_name, max_dummies_num = 10):
    ''' Split values in columns, which contain lists (for example, amenities) '''
    
    # get dictionary of unique values in lists across dataset rows
    values_dict = {}

    for unique_value in df[column_name].unique(): 
        for value in unique_value.replace('[', '').replace("'", '').replace("]", '').replace('"', '').replace('{', '').replace('}', '').split(','):
            if value in values_dict:
                values_dict[value] = values_dict[value] + 1
            else:
                values_dict[value] = 0
                
    values_sorted = sorted(values_dict.items(), key=lambda kv: kv[1], reverse = True)
      
    # split into columns
    for value in values_sorted[: max_dummies_num]:
        df[column_name + '_' + value[0]] = df.apply(lambda row: get_val_from_list(row, column_name, value[0]),axis=1)
        
    return

def get_extra_people_fee(row):
    ''' Return 1 when the is fee for exatra people '''
    if row['extra_people'] == '$0.00':
        return 0.0
    else:
        return 1.0

#Main dataset cleaning function
def clean_dataset(listings_df):
    '''
    INPUT
    listings_df - pandas dataframe containing listings data 
        
    OUTPUT
    df - cleaned dataset, which contains merged tables:
    1. irrelevant columns are dropped;
    2. string containing dates are converted into numbers;
    3. columns, containing lists, are split into several columns (for example, amenities)
    4. missing values are imputed with mean or mode.
    '''
   
    #drop the irrelevant columns
    columns_to_drop = ['host_id','host_location','host_acceptance_rate','host_neighbourhood',
                   'host_total_listings_count', 'weekly_price', 'monthly_price',
                   'security_deposit', 'cleaning_fee', 'calendar_updated',
                   'listing_url','last_scraped' ,'scrape_id', 'name', 'summary', 'space', 'description',
                   'experiences_offered', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'zipcode',
                   'neighborhood_overview', 'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
                   'xl_picture_url', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url', 'host_picture_url',
                   'city', 'state', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude',
                   'is_location_exact', 'square_feet', 'has_availability', 'neighbourhood_group_cleansed',
                   'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped',
                  'first_review', 'last_review', 'requires_license', 'license', 'jurisdiction_names',
                  'reviews_per_month']
    df = listings_df.drop(columns = columns_to_drop) 
     
    #convert price to number
    df['price'] = df['price'].str.replace(r'\$','').str.replace(r'\,','').astype(float)
    
    #convert host_since date into number and fill in missing values, drop the original column
    df['host_since_year'] = df.apply(lambda row: get_host_since_year(row),axis=1)
    df['host_since_year'].fillna(df['host_since_year'].mean(), inplace = True)
    df = df.drop(columns = ['host_since'])
    
    #convert host_response_rate into number and fill in missing values, drop the original column
    df['host_response_rate_num'] = df['host_response_rate'].astype(str)
    df['host_response_rate_num'] = df['host_response_rate_num'].str.replace("%", "").astype("float")
    df['host_response_rate_num'].fillna(df['host_response_rate_num'].mean(), inplace = True)
    
    df['host_response_rate_buckets'] = pd.qcut(df['host_response_rate_num'], 5, labels=False, duplicates = 'drop')
    
    df = df.drop(columns = ['host_response_rate', 'host_response_rate_num'])
    
    #fill missing values with mean value for host_listings_count
    df['host_listings_count'].fillna(df['host_listings_count'].mean(), inplace = True)
    
    #split host_verifications into dummy columns and drop the original column
    split_list_into_columns(df, 'host_verifications')
    df = df.drop(columns = ['host_verifications'])
    
    #fill in missing values for bathrooms, bedrooms and beds with mode
    df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].mode()[0])
    df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].mode()[0])
    df['beds'] = df['beds'].fillna(df['beds'].mode()[0])
    
    #split amenities into dummy columns and drop the original column
    split_list_into_columns(df, 'amenities')
    df = df.drop(columns = ['amenities'])
    
    #turn extra people fee into binary column (1 - if fee for extra people is charged, 0 - otherwise)
    df['extra_people_fee'] = df.apply(lambda row: get_extra_people_fee(row),axis=1)
    df = df.drop(columns = ['extra_people'])
    
    #fill missing values for review scores columns
    review_scores_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                         'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                        'review_scores_value']
    for column in review_scores_columns:
        df[column].fillna(df[column].mean(), inplace = True)
    
    return df

In [4]:
# apply functions above to clean dataset
df = clean_dataset(listings_df)

In [5]:
df.columns.tolist()

['id',
 'host_response_time',
 'host_is_superhost',
 'host_listings_count',
 'host_has_profile_pic',
 'host_identity_verified',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'price',
 'guests_included',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'cancellation_policy',
 'require_guest_profile_picture',
 'require_guest_phone_verification',
 'calculated_host_listings_count',
 'host_since_year',
 'host_response_rate_buckets',
 'host_verifications_email',
 'host_verifications_ phone',
 'host_verifications_ reviews',
 'host_verifications_ jumio',
 'host_verifications_ facebook',
 'host_verifications_ kba',
 'host_verifications_ linkedin',
 'host_verifications_ google',
 'host_verifications_phon

In [6]:
def lr_rf(X, y, TEST_SIZE, RAND_STATE):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RAND_STATE)
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    MSE = mean_squared_error(y_test, predictions)
    Error_lr = round(math.sqrt(MSE),2)
    r2 = model.score(X_test, y_test)
    r2a_lr = 1 - (1-model.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    print('Linear Regression Results:')
    print(f''' MSE: {MSE}, \n R2: {r2}, \n Radjusted: {r2a_lr} \n Error: {Error_lr} \n \n''')
    print('----------------------------------------')
    regressor = RandomForestRegressor(n_estimators=2000, random_state=RAND_STATE)  
    regressor.fit(X_train, y_train)  
    y_pred = regressor.predict(X_test)  
    Error_rf = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    print('Random Forest Results:')
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 
    print(f'R: {regressor.score(X_test, y_test)}')
    r2a_rf = 1 - (1-regressor.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    print(f'RAdjusted: {r2a_rf}')
    print(f'Error: {Error_rf}')

In [7]:
#find minimum, maximum and average price for listing
min_price = df['price'].min()
max_price = df['price'].max()
mean_price = df['price'].mean()

print(f'Minimum price per listing is ${min_price}.')
print(f'Maximum price per listing is ${max_price}.')
print(f'Average price per listing is ${mean_price}.')

Minimum price per listing is $10.0.
Maximum price per listing is $2822.0.
Average price per listing is $149.16599516518937.


In [8]:
df = df[df['price'] <= 699]
df['price'].max()


695.0

In [9]:
df = df[df['minimum_nights'] <= 31]
df['minimum_nights'].max()

30

In [10]:
golden_df = df.dropna()
golden_df.shape

(3274, 54)

In [11]:
#creating CSV file of the improved golden version 

golden_df.to_csv('goldenimproved.csv')

In [12]:
#turn categorical columns into dummies
cat_columns = list(golden_df.select_dtypes(include=['object']).columns)
    
for col in  cat_columns:
    golden_df = pd.concat([golden_df.drop(col, axis=1), pd.get_dummies(golden_df[col], prefix=col, prefix_sep='_',

                                                         drop_first=True, dummy_na=True)], axis=1)


In [13]:
X = golden_df.drop(columns = ['price', 'id'])
y = golden_df["price"]
print(X.shape, y.shape)

(3274, 80) (3274,)


In [14]:
lr_rf(X, y, 0.3, 42)

Linear Regression Results:
 MSE: 2835.1475881994365, 
 R2: 0.6511192328188828, 
 Radjusted: 0.6201763709846374 
 Error: 53.25 
 

----------------------------------------
Random Forest Results:
Mean Absolute Error: 32.632378510762685
Mean Squared Error: 2634.6259800756975
Root Mean Squared Error: 51.32860781353511
R: 0.6757945381785699
RAdjusted: 0.6470401734937424
Error: 51.32860781353511
